In [1]:
import numpy as np
import pandas as pd
import requests
import re
import json
from os import walk
from multiprocessing.pool import ThreadPool

In [2]:
URL = "https://s3.amazonaws.com/irs-form-990/index_2018.json"

response = requests.get(URL)
with open("index_2018.json", 'wb') as f:
    f.write(response.content)

In [3]:
with open("index_2018.json") as f:
    data = json.load(f)
    data = data[list(data.keys())[0]]

In [4]:
df = pd.DataFrame.from_dict(data)
df.tail(5)

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
457505,391455942,201712,93493326005008,990,https://s3.amazonaws.com/irs-form-990/20180326...,ROAD RUNNERS CLUB OF AMERICA 1174 PACE SETTERS...,2018-12-19,201803269349300500,2019-02-21T16:25:33
457506,112808943,201712,93492312011058,990EZ,https://s3.amazonaws.com/irs-form-990/20180312...,UNITED HOMES FUND INC CO FLUSHING HOUSE,2018-12-03,201803129349201105,2019-02-21T16:25:33
457507,262980099,201712,93492311010758,990EZ,https://s3.amazonaws.com/irs-form-990/20180311...,HOUGHTON VOLUNTEER AMBULANCE SERVICE INC,2018-12-03,201803119349201075,2019-02-21T16:25:33
457508,450392710,201806,93493311012808,990,https://s3.amazonaws.com/irs-form-990/20180311...,VALLEY MEMORIAL FOUNDATION,2018-11-30,201803119349301280,2019-02-21T16:25:33
457509,526038675,201712,93493311025608,990,https://s3.amazonaws.com/irs-form-990/20180311...,PLUMBERS AND STEAMFITTERS PROTECTIVE ASSOCIATI...,2018-12-03,201803119349302560,2019-02-21T16:25:33


In [5]:
df.shape

(457510, 9)

In [6]:
def download_url(url):
    file_name_start = url.rfind('/') + 1
    file_name = url[file_name_start:]
    output_dir = "/home/meso/git_repo/exploring_990/data_2018/"
    file_name = output_dir + file_name
    
    r = requests.get(url, stream=True)
    if r.status_code == requests.codes.ok:
        with open(file_name, 'wb') as f:
            f.write(r.content)
    return url

urls = []
for i in range(df.shape[0]): 
    urls.append(data[i]['URL'])
    
results = ThreadPool(100).imap_unordered(download_url, urls)

In [7]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2018/'):
    f.extend(filenames)
    break

In [8]:
u = list(df['URL'])
len(u)

457510

In [9]:
file_names_list = []

for i in range(len(u)):
    file_loc = u[i].rfind("/") + 1
    file_name = u[i][file_loc:]
    file_names_list.append(file_name)

In [10]:
print(len(file_names_list), str(":"), len(f))

457510 : 456984


In [11]:
diff = list(set(file_names_list) - set(f))
len(diff)

526

In [12]:
file_loc = u[i].rfind("/") + 1
url_pre = u[0][:file_loc]

In [13]:
urls = []
for i in range(len(diff)):
    url = url_pre + diff[i]
    urls.append(url)

In [14]:
for url in urls:
    download_url(url)

In [15]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2018/'):
    f.extend(filenames)
    break

In [16]:
len(f)

457510

In [17]:
diff = list(set(file_names_list) - set(f))
len(diff)

0

In [18]:
def list_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [idx for idx, item in enumerate(seq) if item in seen or seen_add(item)]

In [19]:
print(list_duplicates(file_names_list))

[]


In [20]:
dup_df = df.iloc[list_duplicates(file_names_list)]

In [21]:
dup_ein = pd.DataFrame(dup_df['EIN'])

In [22]:
dup_ein

,EIN


In [23]:
duplicates = pd.DataFrame([])

for dup in range(dup_ein.shape[0]):
    ein = dup_ein.iloc[dup]['EIN']
    duplicates = duplicates.append(df[df['EIN'] == ein])

In [24]:
duplicates

""
